# 天池大赛·印象盐城

## 特征工程

初赛提供2012年1月-2017年10月盐城分车型销量配置数据。
第一阶段需要参赛者预测2017年11月盐城分车型销量数据，第二阶段需要参赛者预测2017年12月盐城分车型销量数据。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
train = pd.read_csv('[new] yancheng_train_20171226.csv', low_memory=False)
test = pd.read_csv('yancheng_testA_20171225.csv')

In [3]:
test.head()

,predict_date,class_id,predict_quantity
0,201711,103507,NaN
1,201711,124140,NaN
2,201711,125403,NaN
3,201711,136916,NaN
4,201711,169673,NaN


### 一、异常值处理

1.1 发现数据里相同车型在同一月份里有两条记录，这里做简单加和

In [4]:
# labels = ['sale_date','class_id','sale_quantity']
labels = ['sale_date','class_id','compartment','type_id','level_id','department_id','TR','gearbox_type','displacement','if_charging',
          'price_level','driven_type_id','fuel_type_id','newenergy_type_id','emission_standards_id','if_MPV_id','if_luxurious_id','power',
          'cylinder_number','engine_torque','car_length','car_width','car_height','total_quality','equipment_quality','rated_passenger',
          'wheelbase','front_track','rear_track']
# trainSale = train[labels]
train = train.groupby(labels).agg('sum').reset_index()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Data columns (total 31 columns):
sale_date                20099 non-null int64
class_id                 20099 non-null int64
compartment              20099 non-null int64
type_id                  20099 non-null int64
level_id                 20099 non-null object
department_id            20099 non-null int64
TR                       20099 non-null object
gearbox_type             20099 non-null object
displacement             20099 non-null float64
if_charging              20099 non-null object
price_level              20099 non-null object
driven_type_id           20099 non-null int64
fuel_type_id             20099 non-null object
newenergy_type_id        20099 non-null int64
emission_standards_id    20099 non-null int64
if_MPV_id                20099 non-null int64
if_luxurious_id          20099 non-null int64
power                    20099 non-null object
cylinder_number          20099 non-null int64
engine_t

1.2 三个功率为81/70的条目里共有两款车

    其中一款车有记载的功率为66和81，所以我们取平均值，令其为73
    
    另一款车有记载的功率为66，70,81和96，取平均值，令其为78

In [5]:
train[train['power']=='81/70']

,sale_date,class_id,compartment,type_id,level_id,department_id,TR,gearbox_type,displacement,if_charging,...,car_width,car_height,total_quality,equipment_quality,rated_passenger,wheelbase,front_track,rear_track,sale_quantity,brand_id
12382,201601,175962,3,2,1,4,5,MT,1.6,L,...,1706,1469,1735,1275,5,2603,1460,1500,8,750
13450,201603,961962,3,2,1,4,5,MT,1.6,L,...,1706,1470,1740,1260,5,2603,1460,1500,5,750
13478,201604,175962,3,2,1,4,5,MT,1.6,L,...,1706,1469,1735,1275,5,2603,1460,1500,32,750


In [6]:
train.loc[12382,'power']=73
train.loc[13478,'power']=73
train.loc[13450,'power']=78
train[['power']] = train[['power']].astype('float32')

1.3 发动机扭矩

    有三个功率是'155/140'，又是上面那三个货
    
    还有一个车型是'-'，给个平均值吧

In [7]:
train[train['engine_torque']=='155/140']

,sale_date,class_id,compartment,type_id,level_id,department_id,TR,gearbox_type,displacement,if_charging,...,car_width,car_height,total_quality,equipment_quality,rated_passenger,wheelbase,front_track,rear_track,sale_quantity,brand_id
12382,201601,175962,3,2,1,4,5,MT,1.6,L,...,1706,1469,1735,1275,5,2603,1460,1500,8,750
13450,201603,961962,3,2,1,4,5,MT,1.6,L,...,1706,1470,1740,1260,5,2603,1460,1500,5,750
13478,201604,175962,3,2,1,4,5,MT,1.6,L,...,1706,1469,1735,1275,5,2603,1460,1500,32,750


In [8]:
train.loc[12382,'engine_torque']=143.5
train.loc[13478,'engine_torque']=143.5
train.loc[13450,'engine_torque']=147.4

In [9]:
train[train['engine_torque']=='-']

,sale_date,class_id,compartment,type_id,level_id,department_id,TR,gearbox_type,displacement,if_charging,...,car_width,car_height,total_quality,equipment_quality,rated_passenger,wheelbase,front_track,rear_track,sale_quantity,brand_id
16287,201612,527765,2,2,5,1,1,AT,0.0,L,...,1630,1518,1360,1050,4,2360,1405,1400,195,236
16656,201701,527765,2,2,5,1,1,AT,0.0,L,...,1630,1518,1360,1050,4,2360,1405,1400,6,236
16657,201701,527765,2,2,5,1,1,AT,0.0,L,...,1630,1518,1360,1058,4,2360,1405,1400,6,236
17011,201702,527765,2,2,5,1,1,AT,0.0,L,...,1630,1518,1360,1050,4,2360,1405,1400,35,236
17379,201703,527765,2,2,5,1,1,AT,0.0,L,...,1630,1518,1360,1050,4,2360,1405,1400,59,236
17747,201704,527765,2,2,5,1,1,AT,0.0,L,...,1630,1518,1360,1050,4,2360,1405,1400,30,236
17748,201704,527765,2,2,5,1,1,AT,0.0,L,...,1630,1518,1360,1058,4,2360,1405,1400,18,236
18137,201705,527765,2,2,5,1,1,AT,0.0,L,...,1630,1518,1360,1050,4,2360,1405,1400,6,236
18138,201705,527765,2,2,5,1,1,AT,0.0,L,...,1630,1518,1360,1058,4,2360,1405,1400,24,236
18514,201706,527765,2,2,5,1,1,AT,0.0,L,...,1630,1518,1360,1050,4,2360,1405,1400,53,236


In [10]:
train['engine_torque'][train['engine_torque']=='-']=201.8
train['engine_torque'] = train['engine_torque'].astype('float32')

C:\Users\yudake\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


### 二、特征处理

2.1 获得年和月

In [ ]:
train['year'] = train['sale_date'].apply(lambda x: int(str(x)[0:4]))
train['mouth'] = train['sale_date'].apply(lambda x: int(str(x)[4:6]))
train.info()

2.2 获得本车上个月的销量

2.1 品牌ID共有42个，进行One-hot编码

In [11]:
train['brand_id'].unique()

array([ 761,  106,   98,  836,   12,  814,  831,  750,  537,  450,  692,
        985,  841,  638,  872,  953,  304,  196,  783,  608,  637,   75,
        923, 1566,  497,  813,  290, 1846,  807,  864,  498,  236,  542,
        512,  294,   49, 1500,  126,  682,   68,   76, 1276], dtype=int64)

In [12]:
Brand_id_dummies = pd.get_dummies(train['brand_id'], prefix='brand_id')
train = train.drop(['brand_id'],axis=1)
train = pd.concat([train, Brand_id_dummies],axis=1)

2.2 厢数有三种取值，进行One-hot编码

In [13]:
train['compartment'].unique()

array([2, 3, 1], dtype=int64)

In [14]:
compartment_dummies = pd.get_dummies(train['compartment'], prefix='compartment')
train = train.drop(['compartment'],axis=1)
train = pd.concat([train, compartment_dummies],axis=1)

2.3 车型类别有四种

In [15]:
train['type_id'].unique()

array([3, 2, 4, 1], dtype=int64)

In [16]:
type_id_dummies = pd.get_dummies(train['type_id'], prefix='type_id')
train = train.drop(['type_id'],axis=1)
train = pd.concat([train, type_id_dummies],axis=1)

2.4 车型级别有六种

In [17]:
train['level_id'].unique()

array(['2', '-', '1', '4', '3', '5'], dtype=object)

In [18]:
level_id_dummies = pd.get_dummies(train['level_id'], prefix='level_id')
train = train.drop(['level_id'],axis=1)
train = pd.concat([train, level_id_dummies],axis=1)

2.5 车型系别有七种

In [19]:
train['department_id'].unique()

array([2, 5, 1, 3, 4, 6, 7], dtype=int64)

In [20]:
department_id_dummies = pd.get_dummies(train['department_id'], prefix='department_id')
train = train.drop(['department_id'],axis=1)
train = pd.concat([train, department_id_dummies],axis=1)

2.6 变速器档位有十种

In [21]:
train['TR'].unique()

array(['6', '4', '5', '8', '7', '0', '5;4', '8;7', '9', '1'], dtype=object)

In [22]:
TR_dummies = pd.get_dummies(train['TR'], prefix='TR')
train = train.drop(['TR'],axis=1)
train = pd.concat([train, TR_dummies],axis=1)

2.7 变速器形式有七种

In [23]:
train['gearbox_type'].unique()

array(['AT', 'MT', 'DCT', 'CVT', 'MT;AT', 'AT;DCT', 'AMT'], dtype=object)

In [24]:
gearbox_type_dummies = pd.get_dummies(train['gearbox_type'], prefix='gearbox_type')
train = train.drop(['gearbox_type'],axis=1)
train = pd.concat([train, gearbox_type_dummies],axis=1)

2.8 排量

    区间缩放

In [25]:
train['displacement'].unique()

array([ 2.4,  3. ,  2. ,  2.5,  1. ,  1.1,  1.2,  1.5,  1.4,  1.6,  1.8,
        2.3,  1.9,  2.7,  2.8,  1.3,  3.6,  3.1,  0. ])

In [26]:
# def displacement2seg(displacement):
#     if displacement < 1:
#         displacement = 0
#     elif displacement <= 1.3:
#         displacement = 1
#     elif displacement == 1.4:
#         displacement = 2
#     elif displacement == 1.5:
#         displacement = 3
#     elif displacement == 1.6:
#         displacement = 4
#     elif displacement == 1.8:
#         displacement = 5
#     elif displacement == 2.0:
#         displacement = 6
#     elif displacement <= 2.5:
#         displacement = 7
#     else:
#         displacement = 8
#     return displacement

# train['displacement'] = train['displacement'].apply(displacement2seg)
train.loc[:,'no_displacement']=0
train['no_displacement'][train['displacement']==0]=1
train['displacement'] = train['displacement'].apply(lambda x: (x-1.0)/(3.6-1.0))
# displacement_dummies = pd.get_dummies(train['displacement'], prefix='displacement')
# train = train.drop(['displacement'],axis=1)
# train = pd.concat([train, displacement_dummies],axis=1)
# train.info()

C:\Users\yudake\Anaconda3\lib\site-packages\ipykernel\__main__.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2.9 是否增压

In [27]:
train['if_charging'].unique()

array(['L', 'T'], dtype=object)

In [28]:
if_charging_dummies = pd.get_dummies(train['if_charging'], prefix='if_charging')
train = train.drop(['if_charging'],axis=1)
train = pd.concat([train, if_charging_dummies],axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Columns: 105 entries, sale_date to if_charging_T
dtypes: float32(2), float64(1), int64(18), object(3), uint8(81)
memory usage: 5.1+ MB


2.10 成交段

In [29]:
train['price_level'].unique()

array(['35-50W', '15-20W', '5WL', '5-8W', '8-10W', '25-35W', '20-25W',
       '10-15W', '50-75W'], dtype=object)

In [30]:
price_level_dummies = pd.get_dummies(train['price_level'], prefix='price_level')
train = train.drop(['price_level'],axis=1)
train = pd.concat([train, price_level_dummies],axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Columns: 113 entries, sale_date to price_level_8-10W
dtypes: float32(2), float64(1), int64(18), object(2), uint8(90)
memory usage: 5.1+ MB


2.11 驱动形式

In [31]:
train['driven_type_id'].unique()

array([1, 2, 3], dtype=int64)

In [32]:
driven_type_id_dummies = pd.get_dummies(train['driven_type_id'], prefix='driven_type_id')
train = train.drop(['driven_type_id'],axis=1)
train = pd.concat([train, driven_type_id_dummies],axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Columns: 115 entries, sale_date to driven_type_id_3
dtypes: float32(2), float64(1), int64(17), object(2), uint8(93)
memory usage: 5.0+ MB


2.12 燃料种类

In [33]:
train['fuel_type_id'].unique()

array(['1', '3', '2', '-', '4'], dtype=object)

In [34]:
fuel_type_id_dummies = pd.get_dummies(train['fuel_type_id'], prefix='fuel_type_id')
train = train.drop(['fuel_type_id'],axis=1)
train = pd.concat([train, fuel_type_id_dummies],axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Columns: 119 entries, sale_date to fuel_type_id_4
dtypes: float32(2), float64(1), int64(17), object(1), uint8(98)
memory usage: 4.9+ MB


2.13 新能源类型

In [35]:
train['newenergy_type_id'].unique()

array([1, 2, 3, 4], dtype=int64)

In [36]:
newenergy_type_id_dummies = pd.get_dummies(train['newenergy_type_id'], prefix='newenergy_type_id')
train = train.drop(['newenergy_type_id'],axis=1)
train = pd.concat([train, newenergy_type_id_dummies],axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Columns: 122 entries, sale_date to newenergy_type_id_4
dtypes: float32(2), float64(1), int64(16), object(1), uint8(102)
memory usage: 4.9+ MB


2.14 排放标准

In [37]:
train['emission_standards_id'].unique()

array([3, 5, 1, 2], dtype=int64)

In [38]:
emission_standards_id_dummies = pd.get_dummies(train['emission_standards_id'], prefix='emission_standards_id')
train = train.drop(['emission_standards_id'],axis=1)
train = pd.concat([train, emission_standards_id_dummies],axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Columns: 125 entries, sale_date to emission_standards_id_5
dtypes: float32(2), float64(1), int64(15), object(1), uint8(106)
memory usage: 4.8+ MB


2.15 是否微客MPV

In [39]:
train['if_MPV_id'].unique()

array([2, 1], dtype=int64)

In [40]:
if_MPV_id_dummies = pd.get_dummies(train['if_MPV_id'], prefix='if_MPV_id')
train = train.drop(['if_MPV_id'],axis=1)
train = pd.concat([train, if_MPV_id_dummies],axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Columns: 126 entries, sale_date to if_MPV_id_2
dtypes: float32(2), float64(1), int64(14), object(1), uint8(108)
memory usage: 4.7+ MB


2.16 是否豪华

In [41]:
train['if_luxurious_id'].unique()

array([1, 2], dtype=int64)

In [42]:
if_luxurious_id_dummies = pd.get_dummies(train['if_luxurious_id'], prefix='if_luxurious_id')
train = train.drop(['if_luxurious_id'],axis=1)
train = pd.concat([train, if_luxurious_id_dummies],axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Columns: 127 entries, sale_date to if_luxurious_id_2
dtypes: float32(2), float64(1), int64(13), object(1), uint8(110)
memory usage: 4.6+ MB


2.17 功率

In [43]:
train['power'].unique()

array([ 123.        ,  190.        ,  108.        ,  109.        ,
        135.        ,   45.        ,   50.        ,   60.        ,
         63.        ,  104.        ,   78.        ,   80.        ,
        144.        ,  167.        ,  211.        ,  250.        ,
         78.69999695,   90.40000153,   96.        ,  118.        ,
        147.        ,  110.        ,  125.        ,  166.        ,
        137.        ,  103.        ,   90.        ,   74.        ,
         86.        ,   89.        ,  105.        ,  126.        ,
         98.        ,   77.        ,   88.        ,  112.        ,
        160.        ,  115.        ,  132.        ,  155.        ,
        150.        ,  180.        ,  171.        ,   83.        ,
         82.        ,   81.58000183,   93.        ,  140.        ,
        121.        ,  130.        ,  162.        ,  213.        ,
         73.        ,  114.        ,  120.        ,  128.        ,
        106.        ,   70.        ,   68.        ,  184.     

In [44]:
train['power'] = MinMaxScaler().fit_transform(train['power'])

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


2.17 缸数

In [45]:
train['cylinder_number'].unique()

array([4, 6, 3, 0], dtype=int64)

In [46]:
cylinder_number_dummies = pd.get_dummies(train['cylinder_number'], prefix='cylinder_number')
train = train.drop(['cylinder_number'],axis=1)
train = pd.concat([train, cylinder_number_dummies],axis=1)

2.18 发动机扭矩

In [47]:
train['engine_torque'] = MinMaxScaler().fit_transform(train['engine_torque'])

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


2.19 车长

In [48]:
train['car_length'] = MinMaxScaler().fit_transform(train['car_length'])

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


2.20 车宽

In [49]:
train['car_width'] = MinMaxScaler().fit_transform(train['car_width'])

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


2.21 车高

In [50]:
train['car_height'] = MinMaxScaler().fit_transform(train['car_height'])

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


2.22 质量

In [51]:
train['total_quality'] = MinMaxScaler().fit_transform(train['total_quality'])

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


2.23 装备质量

In [52]:
train['equipment_quality'] = MinMaxScaler().fit_transform(train['equipment_quality'])

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


2.24 轴距

In [53]:
train['wheelbase'] = MinMaxScaler().fit_transform(train['wheelbase'])

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


2.25 前轮距

In [54]:
train['front_track'] = MinMaxScaler().fit_transform(train['front_track'])

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


2.26 后轮距

In [55]:
train['rear_track'] = MinMaxScaler().fit_transform(train['rear_track'])

C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Users\yudake\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


2.27 额定载客

In [56]:
train['rated_passenger'].unique()

array(['7', '5', '7-8', '6-8', '6-7', '5-8', '5-7', '4-5', '9', '4'], dtype=object)

In [57]:
rated_passenger_dummies = pd.get_dummies(train['rated_passenger'], prefix='rated_passenger')
train = train.drop(['rated_passenger'],axis=1)
train = pd.concat([train, rated_passenger_dummies],axis=1)

2.28 得到月份与年份

In [58]:
train['year'] = train['sale_date'].apply(lambda x: int(str(x)[0:4]))
train['mouth'] = train['sale_date'].apply(lambda x: int(str(x)[4:6]))
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20099 entries, 0 to 20098
Columns: 141 entries, sale_date to mouth
dtypes: float32(2), float64(9), int64(6), uint8(124)
memory usage: 4.8 MB


In [ ]:
labels = ['class_id','compartment','type_id','level_id','department_id','TR','gearbox_type','displacement','if_charging',
          'price_level','driven_type_id','fuel_type_id','newenergy_type_id','emission_standards_id','if_MPV_id','if_luxurious_id','power',
          'cylinder_number','engine_torque','car_length','car_width','car_height','total_quality','equipment_quality','rated_passenger',
          'wheelbase','front_track','rear_track']